In [1]:
import xml.etree.ElementTree as ET
import pandas as pd
import re

#Parse xml tree
tree = ET.parse(r"C:\Users\user\Desktop\Proium\Coding_data\customer1129550_loan1131339_crif_report.xml")
root = tree.getroot()



In [2]:
def CreateDF(root):
    """
    Function that creates DataFrame with Account Type and Combined payment history.
    param root: root of tree
    var loanDetails: list with Acc Type and payment history for single iter
    var trade_paymentHistory:  list of loanDetails for all iter
    returns trade_paymentHistory_DF: final dataframe
    """
    
    loanDetails = []
    trade_paymentHistory = []

    for element in root.iter('LOAN-DETAILS'):

        trade = element.find('ACCT-TYPE').text
        paymentHistory = element.find('COMBINED-PAYMENT-HISTORY').text
        loanDetails = [trade, paymentHistory]
        trade_paymentHistory.append(loanDetails)

    trade_paymentHistory_DF = pd.DataFrame(trade_paymentHistory, columns=['ACCT-TYPE', 'COMBINED-PAYMENT-HISTORY'])

    return(trade_paymentHistory_DF)



In [3]:
def GenerateDPDList(string):
    """
    Method that takes payment history as parameter and extracts DPD from each month and returns them as list 
    var regex : generates str between "," and "/". this is date
    returns match: list of all regex
    """
    
    regex = '(?<=\,)(.*?)(?=\/)'
    match = re.findall(regex, string)
    #print(match, len(match))
    
    return(match)

def FindDPD(lst):
    """
    Method that takes lits from GenerateDPDList and returns list 30+DPD 
    """
    l = []
    for elm in lst:
        if elm not in ['DDD', 'XXX', 'STD', '000']: # ignore strings and 000s
            #print(elm)
            if int(elm) >30: #check for >30
                l.append(int(elm))
    return(l)

#a= FindDPD(GenerateDPDList([]))
#print(FindDPD(GenerateDPDList([])), len(a))

In [4]:
#1) What percentage of trades are with 30+ DPD (more than 30 days past due) among all the trades available?

df = CreateDF(root) #df is DataFrame with Account Type and Combined payment history
#print(df)

dpdMonths = []
totalMonth = []

for i in range(len(df)):
    
    elm = df.loc[i][1]
    if elm is None:
        totalMonth.append(0)
        dpdMonths.append(0)
    
    if elm is not None:
        #print(elm)
        tl = GenerateDPDList(elm) #tl is list of dpd for each trades payment history 
        #print(tl, len(tl))
        totalMonth.append(len(tl)) #list of total months for all trades. logic behind this is there will be months equal to all strings between "," and "/". that is len of list.
        dpd = FindDPD(tl) #list of 30+dpd
        #print(dpd, len(dpd))
        dpdMonths.append(len(dpd)) #list of 30+dpd months
    
df['Total months of Loan tenure'] = totalMonth
df['number of dpd occurance'] = dpdMonths

#print(df)

DF = df.groupby('ACCT-TYPE').agg({'number of dpd occurance': 'sum', 'Total months of Loan tenure' : 'sum'}) # summarise no of month data(total months, 30+dpd months) with trade(loan type)
#print(DF)
DF['Percentage'] = DF['number of dpd occurance']*100/DF['Total months of Loan tenure']


In [5]:
def FindSumOFAllDispersedLoanAmount(root):
    """
    Function takes root of tree as parameter and returns sum of disbursed amount to a customer throught their credit lifecycle.     
    """
    
    Sum = 0
    for time in root.iter('DISBURSED-AMT'):
        Sum += int(time.text.replace(',',''))
    return(Sum)

    #for time in root.iter('SCORE-VALUE'):
    #   print(time.text)

#2)What is the sum of total disbursed amount for all loans  for each customer?
Sum = FindSumOFAllDispersedLoanAmount(root)
print(Sum)

795601


In [6]:
DF2 = df.groupby('ACCT-TYPE').agg({'number of dpd occurance': 'max'})

In [7]:
writer = pd.ExcelWriter(r"C:\Users\user\Desktop\Proium\Coding_data\customer1129550_loan1131339_crif_report.xlsx", engine='xlsxwriter')
DF.to_excel(writer, sheet_name='Q1')
DF2.to_excel(writer, sheet_name='Q3')
writer.save()


In [8]:
DF

,number of dpd occurance,Total months of Loan tenure,Percentage
ACCT-TYPE,,,
Gold Loan,11,58,18.965517
Other,10,36,27.777778
Personal Loan,0,21,0.000000
Property Loan,0,3,0.000000


In [9]:
DF2

,number of dpd occurance
ACCT-TYPE,
Gold Loan,4
Other,10
Personal Loan,0
Property Loan,0
